In [3]:
from prometheus_api_client import PrometheusConnect

prom = PrometheusConnect(url='https://prom-gpu.orai.network/', disable_ssl=True)

prom.all_metrics()

/home/kh4ngptt/anaconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'prom-gpu.orai.network'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[':node_memory_MemAvailable_bytes:sum',
 'ALERTS',
 'ALERTS_FOR_STATE',
 'DCGM_FI_DEV_FB_FREE',
 'DCGM_FI_DEV_FB_USED',
 'DCGM_FI_DEV_GPU_TEMP',
 'DCGM_FI_DEV_MEMORY_TEMP',
 'DCGM_FI_DEV_MEM_CLOCK',
 'DCGM_FI_DEV_NVLINK_BANDWIDTH_TOTAL',
 'DCGM_FI_DEV_PCIE_REPLAY_COUNTER',
 'DCGM_FI_DEV_POWER_USAGE',
 'DCGM_FI_DEV_SM_CLOCK',
 'DCGM_FI_DEV_TOTAL_ENERGY_CONSUMPTION',
 'DCGM_FI_DEV_VGPU_LICENSE_STATUS',
 'DCGM_FI_DEV_XID_ERRORS',
 'DCGM_FI_PROF_DRAM_ACTIVE',
 'DCGM_FI_PROF_GR_ENGINE_ACTIVE',
 'DCGM_FI_PROF_PIPE_TENSOR_ACTIVE',
 'aggregator_discovery_aggregation_count_total',
 'aggregator_unavailable_apiservice',
 'aggregator_unavailable_apiservice_total',
 'apiextensions_openapi_v2_regeneration_count',
 'apiextensions_openapi_v3_regeneration_count',
 'apiserver_admission_controller_admission_duration_seconds_bucket',
 'apiserver_admission_controller_admission_duration_seconds_count',
 'apiserver_admission_controller_admission_duration_seconds_sum',
 'apiserver_admission_step_admission_dur

In [4]:
import json
from datetime import datetime, timedelta
query = """avg_over_time(container_memory_usage_bytes{namespace="gpu-operator"}[1m])"""
avg_query = """avg(avg_over_time(container_cpu_allocation{container!=""}[1m])) by (pod)"""
end_time = datetime.now()
start_time = end_time - timedelta(minutes=60)
avg_step = end_time-start_time
ret = prom.custom_query_range(query=query, start_time=start_time, end_time=end_time, step=60)
# print(len(ret), ret[0])
# avg_ret = prom.custom_query_range(query=query,start_time=start_time, end_time=end_time, step=60)
# with open('text.json','w') as f:
#     json.dump(ret,f,indent=4)
# print(avg_ret)
print(start_time)
print(end_time)
# print(step)
ret


/home/kh4ngptt/anaconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'prom-gpu.orai.network'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


2024-08-20 08:48:53.794416
2024-08-20 09:48:53.794416


[{'metric': {'container': 'gc',
   'endpoint': 'https-metrics',
   'id': '/kubepods.slice/kubepods-burstable.slice/kubepods-burstable-poda3f3a1de_c446_4ae1_aecf_6a4b88ccdf5a.slice/cri-containerd-ff5a167dc1d54d6c88a585afa2c935a94bd3a190bd9b344e6012459f0a020925.scope',
   'image': 'registry.k8s.io/nfd/node-feature-discovery:v0.16.3',
   'instance': '210.211.99.160:10250',
   'job': 'kubelet',
   'metrics_path': '/metrics/cadvisor',
   'name': 'ff5a167dc1d54d6c88a585afa2c935a94bd3a190bd9b344e6012459f0a020925',
   'namespace': 'gpu-operator',
   'node': 'vt-gpu-staking-a100',
   'pod': 'gpu-operator-node-feature-discovery-gc-84c5f6b88f-kptkw',
   'service': 'kube-prometheus-stack-kubelet'},
  'values': [[1724118534, '13017088'],
   [1724118594, '12951552'],
   [1724118654, '12969984'],
   [1724118714, '13111296'],
   [1724118774, '13142016'],
   [1724118834, '13168640'],
   [1724118894, '13004800'],
   [1724118954, '12840960'],
   [1724119014, '12875776'],
   [1724119074, '12910592'],
   [

In [47]:
window = '10m'

def get_start_end(window):
    if window == "today":
        end_time = datetime.now()
        start_time = end_time - timedelta(hours=24)
    elif window == "month":
        end_time = datetime.now()
        start_time = datetime(end_time.year, end_time.month, 1)
    elif window == "lastweek":
        end_time = datetime.now()
        start_time = end_time - timedelta(days=7)
    elif window[-1] == "m":
        end_time = datetime.now()
        start_time = end_time - timedelta(minutes=int(window[:-1]))
    elif window[-1] == "h":
        end_time = datetime.now()
        start_time = end_time - timedelta(hours=int(window[:-1]))
    elif window[-1] == "d":
        end_time = datetime.now()
        start_time = end_time - timedelta(days=int(window[:-1]))
    else:
        start_time, end_time = map(int, window.split(','))
        return start_time, end_time
    print(start_time,end_time)
    return start_time,end_time

start, end = get_start_end(window)
resolution = '1m'

def calculate_sum(data):
    values_list = data.get('values', [])
    
    if not values_list:
        return None
    
    total_sum = 0.0
    
    for timestamp, value in values_list:
        total_sum += float(value)
    
    average = total_sum / len(values_list)
    
    return average

query = """avg(avg_over_time(container_memory_working_set_bytes{container!="", container!="POD",pod="jupyterhub-unmounted-pvcs"}[1m])) by (container_name, container, pod_name, pod, namespace, instance, pod)"""

result = prom.custom_query_range(query=query,start_time=start, end_time=end,step='1m')
# for record in result:
#     print(record['metric'])
#     print(calculate_sum(record))
# with open('text.json','w') as f:
#     json.dump(result,f,indent=4)
# for r in result:
#     print(calculate_sum(r))
result

2024-08-20 10:06:25.030490 2024-08-20 10:16:25.030490


/home/kh4ngptt/anaconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'prom-gpu.orai.network'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[]

In [54]:
sum_query = """avg(avg_over_time(container_memory_working_set_bytes{container!="", container_name!="POD", container!="POD"}[10m])) by (container_name, container, pod_name, pod, namespace, instance, pod)""" 

#'rke2-coredns-rke2-coredns-autoscaler-b49765765-9fhjd' 51631445.33333333
prom.custom_query(query=sum_query)

# with open('text2.json','w') as f:
#     json.dump(sum_result,f,indent=4)

/home/kh4ngptt/anaconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'prom-gpu.orai.network'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[{'metric': {'container': 'alpine-pod',
   'instance': '210.211.99.160:10250',
   'namespace': 'jupyterhub',
   'pod': 'alpine-pod'},
  'values': [[1724059127, '62619648'],
   [1724059727, '62619648'],
   [1724060327, '62619648'],
   [1724060927, '62619648'],
   [1724061527, '62619648'],
   [1724062127, '62619648'],
   [1724062727, '62619648']]},
 {'metric': {'container': 'autoscaler',
   'instance': '210.211.99.160:10250',
   'namespace': 'kube-system',
   'pod': 'rke2-coredns-rke2-coredns-autoscaler-b49765765-9fhjd'},
  'values': [[1724059127, '51631308.8'],
   [1724059727, '51550617.6'],
   [1724060327, '51367321.6'],
   [1724060927, '51543449.599999994'],
   [1724061527, '51528294.400000006'],
   [1724062127, '51528294.39999999'],
   [1724062727, '51415449.6']]},
 {'metric': {'container': 'calico-node',
   'instance': '210.211.99.160:10250',
   'namespace': 'kube-system',
   'pod': 'rke2-canal-9h7zv'},
  'values': [[1724059127, '197063270.39999998'],
   [1724059727, '196208640'],
 